In [7]:
import requests
import pandas as pd
import numpy as np
import joblib
import json

# ---------- CONFIGURATION ----------
SOLCAST_API_KEY = "qMT7wX4xfPnqgxeAZXEwday4sCgDdpIm"
LATITUDE = 23.605234
LONGITUDE = 93.117641
HOURS_TO_FORECAST = 24
TIMEZONE = "Asia/Kolkata"

# File paths for your saved model and tools
INPUT_MODEL_FILE = "solar_prediction_model.pkl"
SCALER_FILE = "scaler.pkl"
MODEL_COLUMNS_FILE = "model_columns.json"
OUTPUT_CSV_FILE = "solar_forecast_next_24_hours.csv"
# ----------------------------------

# --- 1. Load your saved model, scaler, and the required column order ---
try:
    print("Loading model, scaler, and column order...")
    model = joblib.load(INPUT_MODEL_FILE)
    scaler = joblib.load(SCALER_FILE)
    with open(MODEL_COLUMNS_FILE, "r") as f:
        model_columns = json.load(f)
    print("Tools loaded successfully.")
except FileNotFoundError as e:
    raise SystemExit(f"ERROR: A necessary file was not found. {e}\nPlease make sure all required files are in the same directory.")

# --- 2. Fetch the weather forecast from the Solcast API ---
api_url = "https://api.solcast.com.au/data/forecast/radiation_and_weather"

# Dynamically create the parameter string to request exactly what the model needs
output_parameters_string = ','.join(model_columns)

params = {
    "latitude": LATITUDE,
    "longitude": LONGITUDE,
    "hours": HOURS_TO_FORECAST,
    "output_parameters": output_parameters_string,
    "format": "json",
    "period": "PT60M"  # <-- THIS IS THE FIX: Request data in 60-minute intervals
}

print("\nRequesting live forecast data from Solcast API...")
try:
    headers = {"Authorization": f"Bearer {SOLCAST_API_KEY}"}
    response = requests.get(api_url, headers=headers, params=params, timeout=30)
    response.raise_for_status() # Raise an error for bad responses (4xx or 5xx)
    data = response.json()
    forecasts = data.get("forecasts", [])
    if not forecasts:
        raise SystemExit("No forecast data returned from Solcast.")
    
    forecast_df = pd.DataFrame(forecasts)
    print("Forecast data successfully received.")

except requests.exceptions.RequestException as e:
    raise SystemExit(f"Solcast API request failed: {e}")

# --- 3. Prepare the forecast data for the model ---
# a) Convert the timestamp from UTC to your local time (IST)
forecast_df['timestamp_utc'] = pd.to_datetime(forecast_df['period_end'])
forecast_df['timestamp_local'] = forecast_df['timestamp_utc'].dt.tz_convert(TIMEZONE)

# b) Create the feature DataFrame (X) and ensure column order is perfect
try:
    X_forecast = forecast_df[model_columns]
    # Ensure all data is numeric before scaling
    X_forecast = X_forecast.astype(float)
except KeyError as e:
    raise SystemExit(f"ERROR: A required column is missing from the API response: {e}")

# --- 4. Scale the features and make the prediction ---
X_scaled = scaler.transform(X_forecast)
predictions = model.predict(X_scaled)

# --- 5. Format and display the final results ---
results_df = pd.DataFrame({
    'Local_Time': (forecast_df['timestamp_local'] - pd.Timedelta(minutes=30)).dt.strftime('%Y-%m-%d %H:%M'),
    'Predicted_MWh': predictions
})

# Clean up the output: round predictions and set any negative predictions to 0
results_df['Predicted_MWh'] = results_df['Predicted_MWh'].round(4)
results_df.loc[results_df['Predicted_MWh'] < 0, 'Predicted_MWh'] = 0

# Save the results to a CSV file
#results_df.to_csv(OUTPUT_CSV_FILE, index=False)
#print(f"\nForecast saved to '{OUTPUT_CSV_FILE}'")

print("\n--- HOURLY ENERGY FORECAST FOR THE NEXT 24 HOURS ---")
print(results_df.to_string(index=False))



Loading model, scaler, and column order...
Tools loaded successfully.

Requesting live forecast data from Solcast API...
Forecast data successfully received.

--- HOURLY ENERGY FORECAST FOR THE NEXT 24 HOURS ---
      Local_Time  Predicted_MWh
2025-10-20 01:00         0.0184
2025-10-20 02:00         0.0110
2025-10-20 03:00         0.0133
2025-10-20 04:00         0.0124
2025-10-20 05:00         0.2484
2025-10-20 06:00         2.4620
2025-10-20 07:00         5.0352
2025-10-20 08:00        11.2246
2025-10-20 09:00        15.3377
2025-10-20 10:00        16.1241
2025-10-20 11:00        13.7434
2025-10-20 12:00        13.0786
2025-10-20 13:00        10.7705
2025-10-20 14:00         8.7633
2025-10-20 15:00         3.2549
2025-10-20 16:00         0.9188
2025-10-20 17:00         0.3523
2025-10-20 18:00         0.0137
2025-10-20 19:00         0.0144
2025-10-20 20:00         0.0113
2025-10-20 21:00         0.0097
2025-10-20 22:00         0.0400
2025-10-20 23:00         0.0055
2025-10-21 00:00    

In [8]:
results_df.head(24)

,Local_Time,Predicted_MWh
0,2025-10-20 01:00,0.0184
1,2025-10-20 02:00,0.0110
2,2025-10-20 03:00,0.0133
3,2025-10-20 04:00,0.0124
4,2025-10-20 05:00,0.2484
5,2025-10-20 06:00,2.4620
6,2025-10-20 07:00,5.0352
7,2025-10-20 08:00,11.2246
8,2025-10-20 09:00,15.3377
9,2025-10-20 10:00,16.1241
